In [ ]:
!pip install -q pyomo

     |████████████████████████████████| 9.2 MB 8.8 MB/s 
     |████████████████████████████████| 49 kB 5.0 MB/s 


In [ ]:
# we add glpk now
!sudo apt install libglpk-dev python3.8-dev libgmp3-dev
!apt-get install -y -qq glpk-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk40 libgmp-dev libgmpxx4ldbl libgraphblas1 libklu1 libldl2 libmetis5
  libpython3.8 libpython3.8-dev libpython3.8-minimal libpython3.8-stdlib
  librbio2 libspqr2 libsuitesparse-dev libsuitesparseconfig5 libumfpack5
  python3.8 python3.8-minimal
Suggested packages:
  libiodbc2-dev gmp-doc libgmp10-doc libmpfr-dev python3.8-venv python3.8-doc
  binfmt-support
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk-dev libglpk40 libgmp-dev libgmp3-dev libgmpxx4ldbl libgraphblas1
  libklu1 libldl2 libmetis5 libpython3.8 libpython3.8-dev libpython3.8-minimal
  libpython3.8-stdlib librbio2 libspqr2 libsuitesparse-dev
  libsuitesparseconfig5 libumfpack5 python3.8 python3.8-dev python3.8-mini

In [ ]:
import pyomo.environ as pyo

In [ ]:
def BIM_maxmin( costs ):
    model    = pyo.ConcreteModel('BIM')
    
    model.x1 = pyo.Var(within=pyo.NonNegativeReals)
    model.x2 = pyo.Var(within=pyo.NonNegativeReals)
    model.z  = pyo.Var() 

    model.profit    = pyo.Objective( sense= pyo.maximize, expr = model.z )

    model.maxmin = pyo.ConstraintList()
    for (c1,c2) in costs:
        model.maxmin.add( expr = model.z <= c1*model.x1 + c2*model.x2 ) 

    model.silicon   = pyo.Constraint(expr =    model.x1              <= 1000)
    model.germanium = pyo.Constraint(expr =                 model.x2 <= 1500)
    model.plastic   = pyo.Constraint(expr =    model.x1 +   model.x2 <= 1750)
    model.copper    = pyo.Constraint(expr =  4*model.x1 + 2*model.x2 <= 4800)

    return model

In [ ]:
BIM = BIM_maxmin( [[12,9], [11,10], [8, 11]] )
results = pyo.SolverFactory('glpk').solve(BIM)

print('X=({:.1f},{:.1f}) value={:.3f}'.format(
    pyo.value(BIM.x1),
    pyo.value(BIM.x2),
    pyo.value(BIM.z)))

X=(583.3,1166.7) value=17500.000
